In [1]:
import pandas as pd

file_path = "/Users/hazel/Desktop/Passkit/Customer Overview/All Paying Customers - September 2, 2024.csv"
data = pd.read_csv(file_path)
data.head()

,Email,Stripe User ID,Plans,MRR,Activity,Activity Amount,Activity Date,Sign Up Date,Signal,LTC,ENG,MRR.1,Lifetime Spent
0,info-aus@cruising.org,pwu_wRpzmsGGOGlV,PassKit V4 Subscription,39.50,new,39.50,2024-09-02,2024-06-24,opportunity,NaN,0.0,0.51,NaN
1,rob.carroll@nosaxaconsulting.com,pwu_oGK3Ms2TOlT9,PassKit V4 Subscription,154.89,upgrade,2.41,2024-09-02,2023-07-24,opportunity,0.59,0.0,2.01,NaN
2,joshua.liebmann@aim.com.au,pwu_Z309zeXSaGq7,PassKit V4 Subscription,84.28,upgrade,0.36,2024-09-02,2021-09-02,opportunity,0.49,0.0,1.09,NaN
3,info@indianoceanoldham.co.uk,pwu_K7n5bvjY8t8F,PassKit V4 Subscription,40.07,downgrade,-6.52,2024-09-02,2023-02-22,opportunity,0.00,0.0,0.52,NaN
4,david.mallam@ribbyhall.co.uk,pwu_CjEa1KelDG9e,PassKit V4 Subscription,75.46,upgrade,3.60,2024-09-02,2024-01-11,opportunity,0.79,0.0,0.98,NaN


# Compare Churn Rates (Loopy Loyalty vs. Passkit)

In [67]:
import panel as pn
import hvplot.pandas  
import pandas as pd

# Enable Panel extensions
pn.extension()

# Churn rates
loopy_loyalty_churn_rate = 0.050869285254346426
passkit_churn_rate = 0.12903225806451613

# Radio button group for churn type selection
churn_type = pn.widgets.RadioButtonGroup(
    name='Churn Type',
    options=['Loopy Loyalty', 'PassKit'],
    button_type='success'
)

# Data for churn rate comparison
data = pd.DataFrame({
    'Service': ['Loopy Loyalty', 'PassKit'],
    'Churn Rate': [loopy_loyalty_churn_rate, passkit_churn_rate],
    'Color': ['#1f77b4', '#ff7f0e']  # Loopy Loyalty (blue), PassKit (orange)
})

# Function to filter the data based on the selected service
def filter_data(service):
    filtered_data = data[data['Service'] == service]
    return filtered_data.hvplot.bar(
        x='Service', y='Churn Rate', color='Color', 
        title=f'{service} Churn Rate', height=400, width=400
    )

# Function to display churn rate text based on the selection
def display_churn_rate(churn_type):
    if churn_type == 'Loopy Loyalty':
        return f"Loopy Loyalty Churn Rate: {loopy_loyalty_churn_rate:.2%}"
    elif churn_type == 'PassKit':
        return f"PassKit Churn Rate: {passkit_churn_rate:.2%}"

# Loopy Loyalty and PassKit customer factors
ll_factors = pd.DataFrame({
    "MRR": [34.233986, 24.512867, 15, 399],
    "LTC": [1.187487, 0.915436, 0, 3],
    "ENG": [0.511153, 1.515957, 0, 25.91]
}, index=["mean", "std", "min", "max"])

passkit_factors = pd.DataFrame({
    "MRR": [155.551935, 649.434267, 16, 12600],
    "LTC": [0.846178, 0.428601, 0, 1.58],
    "ENG": [0, 0, 0, 0]
}, index=["mean", "std", "min", "max"])

# Summary description for Loopy Loyalty and PassKit customers
summary_text = """
### Customer Summary

**Loopy Loyalty:**  
- Customers tend to have lower MRR (average $34).  
- Slightly higher engagement (ENG) compared to PassKit customers.

**PassKit:**  
- Customers have much higher MRR on average ($155).  
- Engagement score is typically 0.  
- This suggests that while PassKit customers generate more revenue, they might be less engaged, contributing to a higher churn rate.
"""

# Panel layout
layout = pn.Column(
    "# Churn Rate Comparison: Loopy Loyalty vs PassKit",
    churn_type,
    pn.bind(display_churn_rate, churn_type),
    pn.bind(filter_data, churn_type),
    summary_text,
    "### Loopy Loyalty Customer Factors Summary",
    pn.pane.DataFrame(ll_factors, width=500, height=200),
    "### PassKit Customer Factors Summary",
    pn.pane.DataFrame(passkit_factors, width=500, height=200)
)

# Show the layout
layout

Column
    [0] Markdown(str)
    [1] RadioButtonGroup(button_type='success', name='Churn Type', options=['Loopy Loyalty', ...], value='Loopy Loyalty')
    [2] ParamFunction(function, _pane=Markdown, defer_load=False)
    [3] ParamFunction(function, _pane=HoloViews, defer_load=False)
    [4] Markdown(str)
    [5] Markdown(str)
    [6] DataFrame(DataFrame, height=200, sizing_mode='fixed', width=500)
    [7] Markdown(str)
    [8] DataFrame(DataFrame, height=200, sizing_mode='fixed', width=500)

# Churn Driver Analysis (Loopy Loyalty vs Passkit)

In [77]:
import panel as pn
import hvplot.pandas  
import pandas as pd

pn.extension()

# Load the data from the CSV file
data = pd.read_csv("/Users/hazel/Desktop/Passkit/Customer Overview/All Paying Customers - September 2, 2024.csv")

# Adding the churn column based on the 'Activity' field
data['Churn'] = data['Activity'].apply(lambda x: 1 if x in ['downgrade', 'cancelled'] else 0)

# Define customer size based on MRR
def define_customer_size(mrr):
    if mrr < 50:
        return "Small"
    elif 50 <= mrr < 200:
        return 'Medium'
    else:
        return 'Large'

data['Customer_Size'] = data['MRR'].apply(define_customer_size)

# Separating Loopy Loyalty and PassKit customers
ll_customers = data[data['Plans'].str.contains('Loopy Loyalty')].copy()
passkit_customers = data[data['Plans'].str.contains('PassKit')].copy()

# Function to get churn drivers (MRR, LTC, ENG) based on service
def get_churn_drivers(service):
    if service == 'Loopy Loyalty':
        churn_drivers = ll_customers.groupby('Churn')[['MRR', 'LTC', 'ENG']].mean()
    else:
        churn_drivers = passkit_customers.groupby('Churn')[['MRR', 'LTC', 'ENG']].mean()
    
    return pn.pane.DataFrame(churn_drivers, width=400, height=200)

# Function to update customer size distribution based on service and customer size selection
def update_plot(service, customer_size):
    if service == 'Loopy Loyalty':
        distribution = ll_customers[ll_customers['Customer_Size'] == customer_size]['Customer_Size'].value_counts()
        color = 'skyblue'
        title = f'Customer Size Distribution for Loopy Loyalty - {customer_size}'
    else:
        distribution = passkit_customers[passkit_customers['Customer_Size'] == customer_size]['Customer_Size'].value_counts()
        color = 'lightgreen'
        title = f'Customer Size Distribution for PassKit - {customer_size}'
    
    return distribution.hvplot.bar(
        title=title, xlabel='Customer Size', ylabel='Number of Customers', height=400, width=600, color=color
    )

# Function to show summaries based on the selected service
def display_summary(service):
    if service == 'Loopy Loyalty':
        return """
        ### Loopy Loyalty Churn Rates by Customer Size:
        - Large customers: Churn rate of 4.82%
        - Small customers: Churn rate of 5.17%
        - **Observation**: Churn rates are similar for both Large and Small customers in Loopy Loyalty, with Small customers slightly more likely to churn than Large customers.

        ### Loopy Loyalty Churn Drivers:
        - **Non-churning customers (Churn = 0)**:
            - MRR: $34.31
            - LTC (Lifetime): 1.20 (approx. 1 year)
            - ENG (Engagement): 0.51
        - **Churning customers (Churn = 1)**:
            - MRR: $32.86
            - LTC (Lifetime): 0.96 (less than 1 year)
            - ENG: 0.46
        - **Observation**: In Loopy Loyalty, churning customers have slightly lower MRR and engagement than non-churning customers. Their lifetime is also shorter, indicating that customers who churn tend to leave within their first year of service.
        """
    elif service == 'PassKit':
        return """
        ### PassKit Churn Rates by Customer Size:
        - Large customers: Churn rate of 13.66%
        - Small customers: Churn rate of 12.58%
        - **Observation**: PassKit shows significantly higher churn rates for both Large and Small customers compared to Loopy Loyalty. The churn rate for Large customers is slightly higher than that for Small customers, suggesting some challenges in retaining high-value customers.

        ### PassKit Churn Drivers:
        - **Non-churning customers (Churn = 0)**:
            - MRR: $154.47
            - LTC (Lifetime): 0.86
            - ENG (Engagement): 0.00
        - **Churning customers (Churn = 1)**:
            - MRR: $162.83
            - LTC: 0.74
            - ENG: 0.00
        - **Observation**: In PassKit, both churning and non-churning customers show zero engagement (ENG = 0), which is concerning. Additionally, churning customers have higher MRR but shorter lifetimes than non-churning customers, suggesting that high-value customers may not be staying as long as expected. This indicates potential issues with product adoption or service satisfaction, especially for high-value customers.
        """

# Service and customer size selectors
service_selector = pn.widgets.RadioButtonGroup(name='Service Selector', options=['Loopy Loyalty', 'PassKit'], button_type='success')
customer_size_selector = pn.widgets.Select(name='Customer Size', options=['Small', 'Medium', 'Large'], value='Small')

# Panel layout
layout = pn.Column(
    "# Churn Rate and Churn Drivers Dashboard",
    service_selector,
    customer_size_selector,
    pn.bind(update_plot, service_selector, customer_size_selector),  
    pn.bind(get_churn_drivers, service_selector),
    pn.bind(display_summary, service_selector)
)

# Show the layout
layout

Column
    [0] Markdown(str)
    [1] RadioButtonGroup(button_type='success', name='Service Selector', options=['Loopy Loyalty', ...], value='Loopy Loyalty')
    [2] Select(name='Customer Size', options=['Small', 'Medium', ...], value='Small')
    [3] ParamFunction(function, _pane=HoloViews, defer_load=False)
    [4] ParamFunction(function, _pane=DataFrame, defer_load=False)
    [5] ParamFunction(function, _pane=Markdown, defer_load=False)

# Pareto Analysis

In [55]:
import panel as pn
import hvplot.pandas
import pandas as pd
import holoviews as hv 
from holoviews import opts

pn.extension()
hv.extension('bokeh')

# Load the data from the CSV file
data = pd.read_csv("/Users/hazel/Desktop/Passkit/Customer Overview/All Paying Customers - September 2, 2024.csv")

# Separate Loopy Loyalty and PassKit customers
ll_customers = data[data['Plans'].str.contains('Loopy Loyalty')]
passkit_customers = data[data['Plans'].str.contains('PassKit')]

# Pareto analysis function
def pareto_analysis(customers, service_name):
    customers_sorted = customers.sort_values(by='MRR', ascending=False)
    total_revenue = customers_sorted['MRR'].sum()
    customers_sorted['Cumulative_Revenue'] = customers_sorted['MRR'].cumsum()
    customers_sorted['Cumulative_Percentage'] = (customers_sorted['Cumulative_Revenue'] / total_revenue) * 100

    pareto_threshold = 80
    top_customers = customers_sorted[customers_sorted['Cumulative_Percentage'] <= pareto_threshold]
    num_top_customers = len(top_customers)

    return customers_sorted[['Cumulative_Percentage', 'MRR']], num_top_customers

# Radio button for service selection
service_selector = pn.widgets.RadioButtonGroup(
    name='Service Selector', 
    options=['Loopy Loyalty', 'PassKit'],
    button_type='success'
)

# Function to update the Pareto chart based on selected service
def update_pareto_chart(service):
    if service == 'Loopy Loyalty':
        sorted_customers, num_top_customers = pareto_analysis(ll_customers, service)
    else:
        sorted_customers, num_top_customers = pareto_analysis(passkit_customers, service)
    
    pareto_plot = sorted_customers.hvplot.line(
        y='Cumulative_Percentage', title=f'Pareto Analysis: {service} Cumulative Revenue Contribution',
        xlabel='Customers (Ranked by MRR)', ylabel='Cumulative Revenue Percentage',
        height=400, width=600
    )
    
    hline = hv.HLine(80).opts(
        line_color='red', line_dash='dashed', line_width=2
    )
    
    text_annotation = hv.Text(x=len(sorted_customers) * 0.9, y=81, text='80% Threshold').opts(
        text_color='red', fontsize=10
    )
    
    combined_plot = pareto_plot * hline * text_annotation

    info = f"Number of {service} customers contributing to 80% of revenue: {num_top_customers}"
    
    return pn.Column(combined_plot, pn.pane.Markdown(info))

# Summary function for each service
def display_summary(service):
    if service == 'Loopy Loyalty':
        return """
        ### Loopy Loyalty Summary:
        - **Large Customer Base**: Loopy Loyalty requires 1,008 customers to reach 80% of total revenue, indicating a more distributed revenue structure. This suggests that Loopy Loyalty relies heavily on a large number of smaller or medium-sized customers for its revenue.
        - **Revenue Stability**: Loopy Loyalty's broad base of customers means that losing a single customer has a smaller impact on overall revenue, contributing to stability.
        - **Lower Individual Customer Value**: The average MRR per customer is lower, implying lower revenue concentration among a few high-value accounts.
        - **Possible Retention Strategy**: Loopy Loyalty's strategy likely focuses on acquiring and retaining a large number of small and medium customers. Improving engagement and upselling could help increase their MRR and reduce churn.
        """
    elif service == 'PassKit':
        return """
        ### PassKit Summary:
        - **Highly Concentrated Revenue**: PassKit requires only 167 customers to contribute to 80% of its total revenue. This indicates that PassKit relies on a smaller number of high-value customers to drive most of its revenue.
        - **Revenue Stability**: PassKit's revenue is more concentrated, meaning losing a few high-value customers could significantly impact overall revenue.
        - **Higher Individual Customer Value**: The smaller number of customers contributing to 80% of revenue suggests that each customer has a much higher MRR, likely concentrated among large customers.
        - **Possible Retention Strategy**: PassKit should focus on retaining high-MRR customers through personalized support, engagement, and proactive account management to prevent churn.
        """

# Main layout with service selector, Pareto chart, and summary
layout = pn.Column(
    "# Pareto Analysis and Business Strategy Summary: Loopy Loyalty vs PassKit",
    service_selector,
    pn.bind(update_pareto_chart, service_selector),
    pn.bind(display_summary, service_selector)
)

# Display the layout
layout

Column
    [0] Markdown(str)
    [1] RadioButtonGroup(button_type='success', name='Service Selector', options=['Loopy Loyalty', ...], value='Loopy Loyalty')
    [2] ParamFunction(function, _pane=Column, defer_load=False)
    [3] ParamFunction(function, _pane=Markdown, defer_load=False)

# Customer size Distribution (Loopy Loyalty vs Passkit)

In [60]:
import panel as pn
import hvplot.pandas  
import pandas as pd

pn.extension()

# Load the data from the CSV file
data = pd.read_csv("/Users/hazel/Desktop/Passkit/Customer Overview/All Paying Customers - September 2, 2024.csv")

# Add churn column based on activity
data['Churn'] = data['Activity'].apply(lambda x: 1 if x in ['downgrade', 'cancelled'] else 0)

# Define customer size based on MRR
def define_customer_size(mrr):
    if mrr < 50:
        return "Small"
    elif 50 <= mrr < 200:
        return 'Medium'
    else:
        return 'Large'

data['Customer_Size'] = data['MRR'].apply(define_customer_size)

# Separate Loopy Loyalty and PassKit customers
ll_customers = data[data['Plans'].str.contains('Loopy Loyalty')].copy()
passkit_customers = data[data['Plans'].str.contains('PassKit')].copy()

# Function to get churn drivers (MRR, LTC, ENG) based on service
def get_churn_drivers(service):
    if service == 'Loopy Loyalty':
        churn_drivers = ll_customers.groupby('Churn')[['MRR', 'LTC', 'ENG']].mean()
    else:
        churn_drivers = passkit_customers.groupby('Churn')[['MRR', 'LTC', 'ENG']].mean()
    
    return pn.pane.DataFrame(churn_drivers, width=400, height=200)

# Function to update customer size distribution plot based on service
def update_plot(service, customer_size):
    if service == 'Loopy Loyalty':
        distribution = ll_customers['Customer_Size'].value_counts()
        color = 'skyblue'
        title = 'Customer Size Distribution for Loopy Loyalty'
    else:
        distribution = passkit_customers['Customer_Size'].value_counts()
        color = 'lightgreen'
        title = 'Customer Size Distribution for PassKit'
    
    return distribution.hvplot.bar(title=title, xlabel='Customer Size', ylabel='Number of Customers', height=400, width=600, color=color)

# Summary text based on selected service
def display_summary(service):
    if service == 'Loopy Loyalty':
        return """
        ### Customer Size Distribution for Loopy Loyalty:
        - **Small Customers**: 1,200 Customers
        - **Medium Customers**: 352 Customers
        - **Large Customers**: 1 Customer

        #### Implications:
        - **Revenue Stability**: Loopy Loyalty's broad base of small customers provides stability since losing one small customer has a relatively minor impact on overall revenue.
        - **Upselling Opportunity**: With only 1 large customer, Loopy Loyalty has a significant opportunity to upsell Small and Medium customers to increase their MRR and push them into higher-tier accounts.
        """
    elif service == 'PassKit':
        return """
        ### Customer Size Distribution for PassKit:
        - **Small Customers**: 477 Customers
        - **Medium Customers**: 127 Customers
        - **Large Customers**: 78 Customers

        #### Observations:
        - **More Balanced Distribution**: PassKit has a more balanced customer size distribution compared to Loopy Loyalty. While small customers still make up the majority, there are significantly more medium and large customers in comparison to Loopy Loyalty.
        - **Larger Base of Large Customers**: With 78 large customers, PassKit has a substantial portion of high-value accounts compared to Loopy Loyalty, which has only 1 large customer.

        #### Implications:
        - **Revenue Concentration**: PassKit's large customers likely contribute a significant portion of the overall revenue, given their higher MRR. However, this also makes PassKit more vulnerable to churn among these large accounts, as losing a single large customer could have a significant impact on revenue.
        - **Potential for Growth Among Medium Customers**: PassKit has a strong base of medium customers who could potentially be upsold to large accounts through targeted retention and engagement strategies.
        """

# Service and customer size selectors
service_selector = pn.widgets.RadioButtonGroup(name='Service Selector', options=['Loopy Loyalty', 'PassKit'], button_type='success')
customer_size_selector = pn.widgets.Select(name='Customer Size', options=['Small', 'Medium', 'Large'], value='Small')

# Panel layout
layout = pn.Column(
    "# Churn Rate and Churn Drivers Dashboard",
    service_selector,
    customer_size_selector,
    pn.bind(update_plot, service_selector, customer_size_selector),
    pn.bind(get_churn_drivers, service_selector),
    pn.bind(display_summary, service_selector)
)

# Show the layout
layout

Column
    [0] Markdown(str)
    [1] RadioButtonGroup(button_type='success', name='Service Selector', options=['Loopy Loyalty', ...], value='Loopy Loyalty')
    [2] Select(name='Customer Size', options=['Small', 'Medium', ...], value='Small')
    [3] ParamFunction(function, _pane=HoloViews, defer_load=False)
    [4] ParamFunction(function, _pane=DataFrame, defer_load=False)
    [5] ParamFunction(function, _pane=Markdown, defer_load=False)

# Upselling and Retention strategy (Loopy Loyalty vs Passkit)

In [65]:
import panel as pn
import hvplot.pandas  
import pandas as pd

pn.extension()

# Load the data from the CSV file
data = pd.read_csv("/Users/hazel/Desktop/Passkit/Customer Overview/All Paying Customers - September 2, 2024.csv")

# Add churn column based on activity
data['Churn'] = data['Activity'].apply(lambda x: 1 if x in ['downgrade', 'cancelled'] else 0)

# Define customer size based on MRR
def define_customer_size(mrr):
    if mrr < 50:
        return "Small"
    elif 50 <= mrr < 200:
        return 'Medium'
    else:
        return 'Large'

data['Customer_Size'] = data['MRR'].apply(define_customer_size)

# Separate Loopy Loyalty and PassKit customers
ll_customers = data[data['Plans'].str.contains('Loopy Loyalty')].copy()
passkit_customers = data[data['Plans'].str.contains('PassKit')].copy()

# Function to get engaged small and medium Loopy Loyalty customers
def get_engaged_small_medium_loopy():
    engaged_small_medium_loopy = ll_customers[
        (ll_customers['Customer_Size'].isin(['Small', 'Medium'])) & 
        (ll_customers['ENG'] > 0.5)
    ]
    return pn.pane.DataFrame(engaged_small_medium_loopy[['MRR', 'Activity', 'ENG']], width=400, height=200)

# Function to get potential large PassKit customers
def get_potential_large_passkit():
    potential_large_customers_passkit = passkit_customers[
        (passkit_customers['Customer_Size'] == 'Medium') & 
        (passkit_customers['ENG'] > 0.5)
    ]
    return pn.pane.DataFrame(potential_large_customers_passkit[['MRR', 'Activity', 'ENG']], width=400, height=200)

# Service selector for dynamic updates
service_selector = pn.widgets.RadioButtonGroup(name='Service Selector', options=['Loopy Loyalty', 'PassKit'], button_type='success')

# Function to dynamically update the upselling table based on the selected service
def update_upselling_table(service):
    if service == 'Loopy Loyalty':
        return get_engaged_small_medium_loopy()
    else:
        return get_potential_large_passkit()

# Adding summary based on the service selection
def display_summary(service):
    if service == 'Loopy Loyalty':
        return """
        ### Loopy Loyalty Upselling Opportunities:
        - **Target Audience**: Small and Medium customers with high engagement (ENG > 0.5).
        - **Opportunity**: These customers are potential candidates for upselling to larger plans, as they are already engaged.
        """
    elif service == 'PassKit':
        return """
        ### PassKit Upselling Opportunities:
        - **Target Audience**: Medium customers with high engagement (ENG > 0.5).
        - **Opportunity**: These customers could be upsold to large accounts by focusing on enhancing their service satisfaction and engagement.
        """

# Panel layout
layout = pn.Column(
    "# Upselling and Retention Strategy",
    service_selector,
    pn.bind(update_upselling_table, service_selector),
    pn.bind(display_summary, service_selector)
)

# Show the layout
layout

Column
    [0] Markdown(str)
    [1] RadioButtonGroup(button_type='success', name='Service Selector', options=['Loopy Loyalty', ...], value='Loopy Loyalty')
    [2] ParamFunction(function, _pane=DataFrame, defer_load=False)
    [3] ParamFunction(function, _pane=Markdown, defer_load=False)